# VacationPy
Using the weather and city data from WeatherPy, create heat maps and other graphs to help plan your next vacation!

In [1]:
# import dependences 

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key

In [2]:
# read in the city csv from WeatherPy

cities = pd.read_csv("burns_cities.csv")
cities.head()

,City_ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qandala,11.47,49.87,78.42,68,15,5.23,SO,1604787043
1,1,ascension,30.22,-90.90,73.40,78,40,10.25,US,1604787043
2,2,chokurdakh,70.63,147.92,1.27,94,100,5.97,RU,1604787043
3,3,brae,60.40,-1.35,44.60,100,32,2.24,GB,1604787043
4,4,ambatondrazaka,-17.83,48.42,59.09,92,66,2.44,MG,1604787043


# Heat Map
Create a heat map that displays the humidity for every city from Part I

In [3]:
humidity = cities["Humidity"].astype(float)
max_humidity = humidity.max()

print(max_humidity)
city_locations = cities[["Latitude", "Longitude"]]

100.0


In [4]:
# configure heat map

city_locations = cities[["Latitude", "Longitude"]]
gmaps.configure(api_key=g_key)
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity,
                                 max_intensity=max_humidity,
                                 dissipating=False,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Narrow Down the Data Frame
Narrow down the DataFrame to find your ideal weather condition. For example:

A max temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.
Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [5]:
narrow_df = pd.DataFrame(cities, columns = ["City", "Country",'Latitude','Longitude', "Temperature", "Wind Speed", "Cloudiness"])

temp = (narrow_df["Temperature"] <= 100) & (narrow_df["Temperature"] > 60)
wind = narrow_df["Wind Speed"] <= 10
cloudiness = narrow_df["Cloudiness"] == 0

df = narrow_df[temp & wind & cloudiness]
df.head()

,City,Country,Latitude,Longitude,Temperature,Wind Speed,Cloudiness
14,castro,BR,-24.79,-50.01,66.29,3.51,0
33,narasannapeta,IN,18.42,84.05,69.93,6.13,0
46,saint-philippe,RE,-21.36,55.77,73.40,6.93,0
68,susa,TN,35.83,10.64,64.40,6.93,0
93,biltine,TD,14.53,20.92,75.25,7.29,0


# Find and Plot Hotels

Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [6]:
#create hotel data frame 

hotel_df=pd.DataFrame(df)
hotel_df['Hotel Name'] = ""

hotel_df.head()

,City,Country,Latitude,Longitude,Temperature,Wind Speed,Cloudiness,Hotel Name
14,castro,BR,-24.79,-50.01,66.29,3.51,0,
33,narasannapeta,IN,18.42,84.05,69.93,6.13,0,
46,saint-philippe,RE,-21.36,55.77,73.40,6.93,0,
68,susa,TN,35.83,10.64,64.40,6.93,0,
93,biltine,TD,14.53,20.92,75.25,7.29,0,


In [7]:
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"radius": 5000, "types": "lodging", "key": g_key}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    params["location"]=f"{row['Latitude']},{row['Longitude']}"
    response = requests.get(hotel_url, params=params).json()
    try:
        hotel_name = response['results'][0]['name']
        #Add hotel name to DF
        hotel_df.loc[index,"Hotel Name"]=hotel_name
    except:
        print("There are no hotels within 5000 meters.")
    
hotel_df.head()

There are no hotels within 5000 meters.
There are no hotels within 5000 meters.
There are no hotels within 5000 meters.


,City,Country,Latitude,Longitude,Temperature,Wind Speed,Cloudiness,Hotel Name
14,castro,BR,-24.79,-50.01,66.29,3.51,0,CHACARA BAILLY
33,narasannapeta,IN,18.42,84.05,69.93,6.13,0,G.lingaraju home
46,saint-philippe,RE,-21.36,55.77,73.40,6.93,0,"Chambres d'hôte ""La Trinité"""
68,susa,TN,35.83,10.64,64.40,6.93,0,Hotel Riadh Palms
93,biltine,TD,14.53,20.92,75.25,7.29,0,Hôtel Trois Étoiles de Biltine


In [8]:
# NOTE: Do not change any of the code in this cell - from template in github

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))